In [5]:
import pandas as pd
import numpy as np
import datetime

# Import the haversine function
from haversine import haversine

# Import the great_circle function from the geopy.distance module
from geopy.distance import great_circle

In [6]:
df = pd.read_excel('Morizon_flats_final.xlsx')

In [7]:
df

,Total_Price,Price_per_Square_Meter,Total_living_area,Number_of_Rooms,Offer_Link,Latitude,Longitude,Developer_name,Walls_height,Investment_name,...,Available_from,Type_of_property,Type_of_Contract,Offer_ID,Date_of_last_update,Date_of_publication,Number_of_levels,Bedroom_area,Bathroom_area,Description
0,607 869 zł,"8608,82","70,61",4.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.038661,19.996587,HSD Inwestycje Kraków,NaN,Bagry,...,NaN,NaN,NaN,morizon-31,28 grudnia 2022,10 września 2020,NaN,NaN,NaN,Promocyjna cena!\n\nNajważniejsze atuty \n✅ No...
1,Zapytaj o cenę,NaN,"42,57",2.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.045432,19.997590,Nexity Polska Sp. z o.o.,NaN,City Vibe,...,NaN,NaN,NaN,morizon-CV-CS-LM-2.B.60,28 grudnia 2022,24 września 2021,NaN,NaN,NaN,Lokal mieszkalny - 2 pokoje z aneksem kuchenny...
2,707 597 zł,"11 860,50","59,66",3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.044608,19.986768,ASA Development Sp z o.o 3 Sp. k.,NaN,Myśliwska Solar Garden,...,NaN,NaN,NaN,morizon-12A,23 listopada 2022,29 lipca 2021,NaN,NaN,NaN,Myśliwska Solar Garden – najważniejsze atuty \...
3,783 900 zł,"12 752,56","61,47",3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.040700,19.999100,NaN,NaN,NaN,...,NaN,własność,na wyłączność,morizon-27,20 grudnia 2022,3 października 2022,NaN,NaN,NaN,"Sprzedam bezpośrednio nowe, gotowe do wprowadz..."
4,Zapytaj o cenę,NaN,"41,08",2.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.037360,19.975432,Activ Investment Sp. z o.o.,NaN,Staw Płaszowski,...,NaN,NaN,NaN,morizon-2,9 grudnia 2022,3 września 2022,NaN,NaN,NaN,"Najważniejsze atuty \n✅ Kameralny, niewysoki b..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9646,550 000 zł,"9369,68","58,70",3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.987460,19.902575,NaN,NaN,NaN,...,NaN,Odrębna własność lokalu,NaN,morizon-ECH-MS-16749,6 grudnia 2022,28 listopada 2022,NaN,2.0,NaN,| Na sprzedaż mieszkanie 3 pokojowe (59 m2) z ...
9647,470 000 zł,"9038,46","52,00",3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.996600,19.936100,NaN,NaN,NaN,...,NaN,własność,na wyłączność,morizon-233,21 listopada 2022,21 listopada 2022,1.0,NaN,NaN,Kraków - ul. Borowinowa : 470 000 złotych (do ...
9648,449 000 zł,8980,"50,00",2.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.999075,19.992177,NaN,NaN,NaN,...,NaN,NaN,NaN,morizon-SPEK-MS-187,dzisiaj,2 października 2022,NaN,NaN,NaN,LOKALIZACJA Budynek znajduje się w zielonej i ...
9649,650 000 zł,"9701,49","67,00",3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.984342,19.896759,NaN,NaN,NaN,...,NaN,Własność,NaN,morizon-KNP-MS-86641,16 grudnia 2022,11 lutego 2022,NaN,2.0,NaN,Premium Estates ma przyjemność zaprezentować m...


In [8]:
# Remove rows where the price is not available
df = df[df.Total_Price != 'Zapytaj o cenę']

# Keep only rows with number of rooms information
df = df.dropna(subset=["Number_of_Rooms"])

# Drop unnecessary columns
columns_to_drop = ["Developer_name", "Walls_height", "Investment_name", "Kitchen_type", "Number_of_bathrooms", "Toilet_and_WC_type", "Total_area_of_Balcony", "Total_area_of_Garden", "Available_from", "Type_of_property", "Type_of_Contract", "Number_of_levels", "Bedroom_area", "Bathroom_area", "Offer_ID", "Description"]
df = df.drop(columns=columns_to_drop)

# Replace "parter" with 0 in the "Floor" column and keep only the first part of the string
df["Floor"] = df["Floor"].str.replace("parter", "0")
df["Floor"] = df["Floor"].str.split("/").str[0]

In [9]:
# Remove currency symbols and spaces from the Total_Price column
df["Total_Price"] = df["Total_Price"].str.replace(" zł", "")
df["Total_Price"] = df["Total_Price"].str.replace(".", "")
df["Total_Price"] = df["Total_Price"].str.replace(" ", "")
df["Total_Price"] = df["Total_Price"].str.replace("zł", "")
df["Total_Price"] = df["Total_Price"].str.replace("~", "")

# Convert Total_Price to float
df['Total_Price'] = df['Total_Price'].astype(float)

# Remove spaces from the Price_per_Square_Meter column and convert it to float
df["Price_per_Square_Meter"] = df["Price_per_Square_Meter"].str.replace(",", ".")
df["Price_per_Square_Meter"] = df["Price_per_Square_Meter"].str.replace(" ", "")
df['Price_per_Square_Meter'] = df['Price_per_Square_Meter'].astype(float)

# Convert Total_living_area to float
df["Total_living_area"] = df["Total_living_area"].str.replace(",", ".")
df['Total_living_area'] = df['Total_living_area'].astype(float)

# Convert Floor and Total_number_of_floors to float
df['Floor'] = df['Floor'].astype(float)
df['Total_number_of_floors'] = df['Total_number_of_floors'].astype(float)

# Replace values in the Balcony column and convert it to float
df['Balcony'] = df['Balcony'].replace({'Tak': 1, '*': 0, 'Nie': 0})
df['Balcony'] = df['Balcony'].fillna(0)
df['Balcony'] = df['Balcony'].astype(float)

# Replace values in the Market_type column and convert it to float
df['Market_type'] = df['Market_type'].replace({'wtórny': 0, 'pierwotny': 1})
df['Market_type'] = df['Market_type'].astype(float)

# Remove spaces from the Date_of_last_update and Date_of_publication columns
df["Date_of_last_update"] = df["Date_of_last_update"].str.replace(" ", ".")
df["Date_of_publication"] = df["Date_of_publication"].str.replace(" ", ".")

C:\Users\Huawei\AppData\Local\Temp\ipykernel_176\3107772982.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Total_Price"] = df["Total_Price"].str.replace(".", "")


In [10]:
def get_todays_date():
    today = datetime.datetime.now()
    return today.strftime('%d.%m.%Y')

def get_yesterdays_date():
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    return yesterday.strftime('%d.%m.%Y')

In [11]:
# Replace "dzisiaj" and "wczoraj" with the current and yesterday's dates, respectively
df["Date_of_last_update"] = df["Date_of_last_update"].str.replace("dzisiaj", get_todays_date())
df["Date_of_publication"] = df["Date_of_publication"].str.replace("dzisiaj", get_todays_date())
df["Date_of_last_update"] = df["Date_of_last_update"].str.replace("wczoraj", get_yesterdays_date())
df["Date_of_publication"] = df["Date_of_publication"].str.replace("wczoraj", get_yesterdays_date())

# Replace month names with their numeric equivalents
month_names = ["stycznia", "lutego", "marca", "kwietnia", "maja", "czerwca", "lipca", "sierpnia", "września", "października", "listopada", "grudnia"]
month_numbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for i in range(len(month_names)):
    df["Date_of_last_update"] = df["Date_of_last_update"].str.replace(month_names[i], month_numbers[i])
    df["Date_of_publication"] = df["Date_of_publication"].str.replace(month_names[i], month_numbers[i])

# Convert the Date_of_last_update and Date_of_publication columns to datetime format
df['Date_of_last_update'] = pd.to_datetime(df['Date_of_last_update'], format='%d.%m.%Y')
df['Date_of_publication'] = pd.to_datetime(df['Date_of_publication'], format='%d.%m.%Y')

In [12]:
# Get today's date
today = datetime.datetime.now().date()

# Calculate the number of days since the last update and publication
df['days from last update'] = today - df['Date_of_last_update'].dt.date
df['days from publication'] = today - df['Date_of_publication'].dt.date

# Convert the number of days to seconds
df['days from last update'] = df['days from last update'].apply(lambda x: x.total_seconds() / 86400)
df['days from publication'] = df['days from publication'].apply(lambda x: x.total_seconds() / 86400)

# Calculate the mean value for the 'Total_number_of_floors' and 'Floor' columns
mean_floors = df['Total_number_of_floors'].mean()
mean_floor = df['Floor'].mean()

# Fill missing values in the 'Total_number_of_floors' and 'Floor' columns with the calculated mean values
df['Total_number_of_floors'] = df['Total_number_of_floors'].fillna(mean_floors)
df['Floor'] = df['Floor'].fillna(mean_floor)

In [13]:
# Remove duplicate rows based on the Offer_Link column
df = df.drop_duplicates(subset='Offer_Link')

In [14]:
df

,Total_Price,Price_per_Square_Meter,Total_living_area,Number_of_Rooms,Offer_Link,Latitude,Longitude,Floor,Total_number_of_floors,Balcony,Market_type,Date_of_last_update,Date_of_publication,days from last update,days from publication
0,607869.0,8608.82,70.61,4.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.038661,19.996587,1.0,5.0,1.0,1.0,2022-12-28,2020-09-10,14.0,853.0
2,707597.0,11860.50,59.66,3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.044608,19.986768,1.0,4.0,1.0,1.0,2022-11-23,2021-07-29,49.0,531.0
3,783900.0,12752.56,61.47,3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.040700,19.999100,1.0,5.0,1.0,1.0,2022-12-20,2022-10-03,22.0,100.0
6,269000.0,7637.71,35.22,2.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.039000,19.941600,4.0,4.0,0.0,0.0,2022-12-30,2020-05-26,12.0,960.0
8,853244.0,11860.50,71.94,4.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,50.044608,19.986768,1.0,4.0,1.0,1.0,2022-09-23,2021-07-29,110.0,531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9646,550000.0,9369.68,58.70,3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.987460,19.902575,3.0,3.0,1.0,0.0,2022-12-06,2022-11-28,36.0,44.0
9647,470000.0,9038.46,52.00,3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.996600,19.936100,2.0,3.0,1.0,0.0,2022-11-21,2022-11-21,51.0,51.0
9648,449000.0,8980.00,50.00,2.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.999075,19.992177,0.0,2.0,0.0,0.0,2023-01-11,2022-10-02,0.0,101.0
9649,650000.0,9701.49,67.00,3.0,https://www.morizon.pl/oferta/sprzedaz-mieszka...,49.984342,19.896759,1.0,3.0,1.0,0.0,2022-12-16,2022-02-11,26.0,334.0


In [15]:
df.to_excel('cleaneddata.xlsx')